In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('DATA\AMES_Final_DF.csv')
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


Seperating sale price (label), rest are features

In [3]:
x = df.drop('SalePrice',axis=1)
y = df['SalePrice']

Using Grid Search Method cross validation approach

In [4]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=101)

Scaling the features

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
scaled_x_train = scaler.transform(x_train)
scaled_x_test = scaler.transform(x_test)

Using Elastic Net model

In [8]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

elastic_net_model = ElasticNet()

#to use the grid method approach, trying different combination hyper parametrs
param_grid = {'alpha':[0.1,1,15,10,50,100],'l1_ratio':[.1,.5,.7,.95,.99,1]}

grid_model = GridSearchCV(estimator=elastic_net_model,param_grid=param_grid,scoring='neg_mean_squared_error',cv=10,verbose=1)
grid_model.fit(scaled_x_train,y_train)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


c:\Users\sai.nikhil.pillai\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.860e+11, tolerance: 1.537e+09
  model = cd_fast.enet_coordinate_descent(
c:\Users\sai.nikhil.pillai\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.817e+11, tolerance: 1.534e+09
  model = cd_fast.enet_coordinate_descent(
c:\Users\sai.nikhil.pillai\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisati

GridSearchCV(cv=10, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 15, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [9]:
grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

Checking the accuracy of the model

In [13]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

y_pred = grid_model.predict(scaled_x_test)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

mae

14195.354900562173

In [14]:
rmse = np.sqrt(mse)
rmse

20558.508566893164

In [15]:
df['SalePrice'].mean()

180815.53743589742

Final Conclusion 

The achieved root mean squared error and the mean absolute error is nealy 10% of the mean sales price. So whatever the sale price the model predicts, the real world price can be either plus 10% or minus 10%